In [5]:
import pandas as pd
import torch
from preprocessing import *
from RNN import RNN

set_seed(42)

In [6]:
df = pd.read_csv(r'output_csv\ANL-Intrepid-2009-1.swf.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 68936 entries, 0 to 68935
Data columns (total 18 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   job_id                    68936 non-null  float64
 1   submit_time               68936 non-null  float64
 2   wait_time                 68936 non-null  float64
 3   run_time                  68936 non-null  float64
 4   num_allocated_processors  68936 non-null  float64
 5   avg_cpu_time_used         68936 non-null  float64
 6   used_memory               68936 non-null  float64
 7   requested_processors      68936 non-null  float64
 8   requested_time            68936 non-null  float64
 9   requested_memory          68936 non-null  float64
 10  status                    68936 non-null  float64
 11  user_id                   68936 non-null  float64
 12  group_id                  68936 non-null  float64
 13  executable_id             68936 non-null  float64
 14  queue_

In [7]:
feature_columns = ['requested_processors', 'requested_time', 'submit_time', 'wait_time', 'user_id', 'queue_id']
target_column = 'run_time'

# Hyperparameters
input_dim = len(feature_columns)
num_hidden = 64
num_layers = 3
dropout = 0.2
num_epochs = 10
lr = 0.001
batch_size = 128  
seq_len = 20
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

dataloaders, scaler = prepare_data_seq(df, feature_columns, target_column, statuss=-1, seq_len=seq_len, batch_size=batch_size)

In [8]:
# Initialize the model
model = RNN(input_dim, num_hidden, num_layers, dropout).to(device)

# Train the model
model.train_model(model, dataloaders['train'], dataloaders['val'], epochs=num_epochs, lr=lr, scaler=scaler)

# Evaluate the model
rmse, mae, mse, r2 = model.evaluate_model(model, dataloaders['test'], scaler, num_features=input_dim)

print(f'\nFinal Test Metrics:')
print(f'Test RMSE: {rmse:.4f}, Test MAE: {mae:.4f}, Test MSE: {mse:.4f}, Test R2: {r2:.4f}')

Epoch [1/10], Train Loss: 0.1213, Val Loss: 0.1402
Val RMSE: 6456.7294, Val MAE: 3077.0321, Val MSE: 41689354.7208, Val R2: 0.6121
Model saved at epoch 1 with validation loss: 0.1402
Epoch [2/10], Train Loss: 0.1056, Val Loss: 0.1541
Val RMSE: 6806.7342, Val MAE: 3191.0203, Val MSE: 46331629.9944, Val R2: 0.5689
Epoch [3/10], Train Loss: 0.1016, Val Loss: 0.1660
Val RMSE: 6992.2717, Val MAE: 3517.6344, Val MSE: 48891863.8086, Val R2: 0.5451
Epoch [4/10], Train Loss: 0.1003, Val Loss: 0.1532
Val RMSE: 6812.0609, Val MAE: 3126.5845, Val MSE: 46404173.1005, Val R2: 0.5682
Epoch [5/10], Train Loss: 0.0985, Val Loss: 0.1635
Val RMSE: 6974.2927, Val MAE: 3452.4460, Val MSE: 48640758.0950, Val R2: 0.5474
Epoch [6/10], Train Loss: 0.0982, Val Loss: 0.1488
Val RMSE: 6720.2735, Val MAE: 3146.5223, Val MSE: 45162075.6423, Val R2: 0.5798
Early stopping triggered

Final Test Metrics:
Test RMSE: 8665.1158, Test MAE: 4976.3650, Test MSE: 75084231.5525, Test R2: 0.3974
